# QR Algorithm

Ci sono delle proprietà:

- Dato che gli eigenvalue di una matrice tirangolare sono gli elementi sulla diagonale, questo algoritmo permette di trovarli.
- Similarity transformation preserva gli eigenvalue.

La matrice A viene **decomposta** in A = QR dove Q è ortogonale e R è triangolare superiore.

In [1]:
import numpy as np
from itertools import product
from math import copysign, hypot, sqrt
from scipy.linalg import hessenberg

def print_matrix(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))
    return

# Shifted QR Algorithm

Questo approccio ci consente di ridurre considerevolmente il tempo di esecuzione dell'algoritmo QR, ma per poterlo utilizzare è necessario ridurre la matrice di partenza in forma di Hessenberg

## Riduzione in forma di Hessenberg
Per ridurre la matrice di partenza A in forma di Hessenberg utilizzeremo le Givens Rotation, per azzerare gli elementi sotto la prima sottodiagonale.
Iteriamo lungo le colonne a partire dall'ultimo elemento, così da poter scegliere come pivot l'elemento immediatamente sopra.

In [20]:
# TODO da controllare se funziona per matrici non quadrate, e in caso implementare una versione funzionante.

def Givens_Rotation_Matrix_Entries(a, b):
    #r = hypot(a.astype(complex), b.astype(complex))
    #r = sqrt(a.astype(complex)**2 + b.astype(complex)**2)
    r = hypot(a,b)
    c = a/r
    s = b/r    
    return c, s

def Hessenberg_reduction(A):
    # mi trovo gli indici della sottodiagonale corrispondente agli elementi da azzerarcole
    (num_rows, num_cols) = np.shape(A)
    #itero sugli indici appena trovati 
    for col in range(num_cols-2):
        for row in reversed(range(2+col,num_cols)):
            # trova gli indici della matrice di givens unando come pivot [col, col](sempre != 0)
            c, s = Givens_Rotation_Matrix_Entries(A[row-1, col], A[row, col])
            #si costruisce la matrice di givens partendo dall'identità
            G = np.identity(num_rows)
            #G.astype(complex)
            #assegna gli indici trovati precedentemente alla matrice di givens
            G[row-1, row-1] = c
            G[row, row] = c
            G[row, row-1] = -s
            G[row-1, row] = s
            # annichilisce l'elemento in posizione A[row, col]
            A = G @ A @ G.T
           
            if A[row, col] < 1.e-16:
                A[row,col] = 0

    return A

In [21]:
A = np.random.random((5, 5))
A = Hessenberg_reduction(A)
print_matrix(A)

(0.7271494893271033+0j)	(0.3950968608750848+0j)	(0.7151900577249689+0j) 	(-0.2323443868862157+0j) 	(0.48672057656756823+0j)  
(1.167474792481163+0j) 	(1.56967795144568+0j)  	(0.9132994161653475+0j) 	(-0.23316122645135556+0j)	(-0.31709550260123315+0j) 
0j                     	(1.2163429396787522+0j)	(0.31675243510088963+0j)	(0.14966193504421405+0j) 	(-0.16873415411031464+0j) 
0j                     	0j                     	(0.15840995387395826+0j)	(0.2064384935119564+0j)  	(-0.003255237008346824+0j)
0j                     	0j                     	0j                      	(0.19296701867242888+0j) 	(-0.5651209208484306+0j)  


## Shifted QR
Ora che abbiamo una matrice in forma di Hessenberg possiamo passare all'implementazione dello Shifted QR algorithm, l'aumento di velocità dipende dal fatto che, se si parte da una matrice in forma di Hessenberg allora sono necessarie solo O(n^2) operazioni.

In particolare noi utilizzeremo una single shift strategy con i Rayleigh quotient shift

In [22]:
def shifted_QR(A):
    H = Hessenberg_reduction(A)
    #H = hessenberg(A)
    #H.astype(complex)
    (num_rows, num_cols) = np.shape(H)
    for m in range(num_rows-1, 0, -1):
        count = 0
        while (abs(H[m,m-1]) > 1.e-20) and 1000 > count:
            count += 1
            Q, R = np.linalg.qr(H - (H[m,m] * np.eye(num_rows)))
            H = (np.dot(R, Q)) + (H[m,m] * np.eye(num_rows))
        H[m,m-1] = 0
    return H

In [23]:
A = np.random.random((5, 5))
print(np.linalg.eig(A)[0])
#A = A.astype(complex)
H = shifted_QR(A)
print_matrix(H)

[ 2.38305437+0.j         -0.29865503+0.37871917j -0.29865503-0.37871917j
  0.33477813+0.36241226j  0.33477813-0.36241226j]


C:\Users\tomma\AppData\Local\Temp\ipykernel_15284\1297531189.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  r = hypot(a,b)
C:\Users\tomma\AppData\Local\Temp\ipykernel_15284\1297531189.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row-1] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_15284\1297531189.py:24: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_15284\1297531189.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row-1] = -s
C:\Users\tomma\AppData\Local\Temp\ipykernel_15284\1297531189.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row] = s


(2.3830543668462925+0j)	(0.24613552852557913+0j)	(-0.5687168432647985+0j) 	(0.06728904834692226+0j)	(-0.10518463180417272+0j)
0j                     	(-0.5307102198173428+0j)	(0.3319992114149355+0j)  	(0.38514188416435957+0j)	(-0.5901605700989215+0j) 
0j                     	0j                      	(-0.06659984540661901+0j)	(0.0871816514981949+0j) 	(0.20528500713855874+0j) 
0j                     	0j                      	0j                       	(0.33477813415722746+0j)	(-0.24785376373008136+0j)
0j                     	0j                      	0j                       	0j                      	(0.33477813415722746+0j) 
